In [ ]:
import numpy as np
import pandas as pd

from PIL import Image
import gc

from datasets import Dataset, load_from_disk

In [ ]:
import yaml
import os
from pathlib import Path

# 1. Get the path of the script
current_file = Path(__file__).resolve() # src/training/your_script.py

# 2. Go up one level to 'src', then into 'config'
config_path = current_file.parent.parent / "config" / "config_general.yaml"

# 3. Load the YAML
with open(config_path, "r") as f:
    config = yaml.safe_load(f)

# 4. Resolve the root of the project (one level above 'src')
# This ensures that "./data" in the YAML is interpreted relative to the Project_Root
PROJECT_ROOT = current_file.parent.parent.parent
os.chdir(PROJECT_ROOT) 

# Extract paths from YAML
DATA_DIR = config['paths']['data']
CHECKPOINT_DIR = config['paths']['checkpoints']
SAM_CHECKPOINT = config['paths']['sam_checkpoint']

In [ ]:
train_dataset = load_from_disk(os.path.join(DATA_DIR, 'datasetTestFinal'))

In [ ]:
#Multiple ids in idx
idx = [0, 1, 2]

In [ ]:
item = train_dataset[idx]
ground_truth_mask = np.array(item["label"])
VH0 = np.array(item["VH0"])
VH1 = np.array(item["VH1"])
VV0 = np.array(item["VV0"])
VV1 = np.array(item["VV1"])
dem = np.array(item["dem"])
slope = np.array(item["slope"])

#Combine all channels to create image
image = np.stack([VH0, VH1, VV0, VV1, dem, slope], axis=1)

In [ ]:
print(image.shape)
print(ground_truth_mask.shape)
print(VH0.shape)
print(VH1.shape)
print(VV0.shape)
print(VV1.shape)
print(dem.shape)
print(slope.shape)

In [ ]:
import matplotlib.pyplot as plt

#draw parts of a single image

singleimage = image[2, :, :, :]

# Plot each channel of the single image
fig, axs = plt.subplots(1, 6, figsize=(15, 5))
axs[0].imshow(singleimage[0, :, :], cmap='gray')
axs[0].set_title('VH0')
axs[1].imshow(singleimage[1, :, :], cmap='gray')
axs[1].set_title('VH1')
axs[2].imshow(singleimage[2, :, :], cmap='gray')
axs[2].set_title('VV0')
axs[3].imshow(singleimage[3, :, :], cmap='gray')
axs[3].set_title('VV1')
axs[4].imshow(singleimage[4, :, :], cmap='gray')
axs[4].set_title('DEM')
axs[5].imshow(singleimage[5, :, :], cmap='gray')
axs[5].set_title('Slope')
plt.tight_layout()
plt.show()

In [ ]:
# get bounding box prompt
# tried -5 5, -5 15, -5 10
prompt = item["box"]
xchange = np.random.randint(-5, 5)
ychange = np.random.randint(-5, 5)
wchange = np.random.randint(-5, 5) 
hchange = np.random.randint(-5, 5) 
input_boxes = []
for box in prompt: 
    x, y, w, h = box
    input_boxes.append([[max(0, x - xchange), max(0, y - ychange), min(512, x + w + wchange), min(512, y + h + hchange)]])

In [ ]:
from transformers import SamProcessor

processor = SamProcessor.from_pretrained("facebook/sam-vit-base")

In [ ]:
# prepare image and prompt for the model
inputs = processor(image[:,0:3,:,:], input_boxes=input_boxes, do_normalize=False, return_tensors="pt", do_rescale=False)

In [ ]:
import torch.nn.functional as F
import torch
image_tensor = torch.tensor(image, dtype=torch.float32)
# Assuming `image_tensor` has shape (batch, 6, 512, 512)
image_resized = F.interpolate(image_tensor, size=(1024, 1024), mode='bilinear', align_corners=False)
print(image_resized.shape)

In [ ]:
singleimage = image_resized[2, :, :, :]

# Plot each channel of the single image
fig, axs = plt.subplots(1, 6, figsize=(15, 5))
axs[0].imshow(singleimage[0, :, :], cmap='gray')
axs[0].set_title('VH0')
axs[1].imshow(singleimage[1, :, :], cmap='gray')
axs[1].set_title('VH1')
axs[2].imshow(singleimage[2, :, :], cmap='gray')
axs[2].set_title('VV0')
axs[3].imshow(singleimage[3, :, :], cmap='gray')
axs[3].set_title('VV1')
axs[4].imshow(singleimage[4, :, :], cmap='gray')
axs[4].set_title('DEM')
axs[5].imshow(singleimage[5, :, :], cmap='gray')
axs[5].set_title('Slope')
plt.tight_layout()
plt.show()

singleimage = image[2, :, :, :]

# Plot each channel of the single image
fig, axs = plt.subplots(1, 6, figsize=(15, 5))
axs[0].imshow(singleimage[0, :, :], cmap='gray')
axs[0].set_title('VH0')
axs[1].imshow(singleimage[1, :, :], cmap='gray')
axs[1].set_title('VH1')
axs[2].imshow(singleimage[2, :, :], cmap='gray')
axs[2].set_title('VV0')
axs[3].imshow(singleimage[3, :, :], cmap='gray')
axs[3].set_title('VV1')
axs[4].imshow(singleimage[4, :, :], cmap='gray')
axs[4].set_title('DEM')
axs[5].imshow(singleimage[5, :, :], cmap='gray')
axs[5].set_title('Slope')
plt.tight_layout()
plt.show()

In [ ]:
import torch
# get bounding box prompt
# tried -5 5, -5 15, -5 10
prompt = item["box"]
xchange = np.random.randint(-5, 5)
ychange = np.random.randint(-5, 5)
wchange = np.random.randint(-5, 5)
hchange = np.random.randint(-5, 5)
input_boxes = []
for box in prompt: 
    x, y, w, h = box
    input_boxes.append([[max(0, x - xchange), max(0, y - ychange), min(512, x + w + wchange), min(512, y + h + hchange)]])

boxes_tensor = torch.tensor(input_boxes, dtype=torch.float)

In [ ]:
print(boxes_tensor.shape)
boxes_tensor2 = (boxes_tensor * 2).squeeze(1) 
print(boxes_tensor2.shape)
print(boxes_tensor2)
print(boxes_tensor)


In [ ]:
#Fileter for full image bboxes only
full_image_indices = []
for i in range(len(train_dataset)):
    box = train_dataset[i]['box']
    if box == [0, 0, 512, 512]:
        full_image_indices.append(i)
train_dataset = train_dataset.select(full_image_indices)

In [ ]:
import sys
import os
# Add the directory containing lit_sam_model.py to the Python path
sys.path.append(os.path.abspath("../"))
from dataprocessing.rcsHandlingFunctions import _rescale
from matplotlib import pyplot as plt

item = train_dataset[22]
ground_truth_mask = np.array(item["label"])
VH0 = np.array(item["VH0"])
VH1 = np.array(item["VH1"])
VV0 = np.array(item["VV0"])
VV1 = np.array(item["VV1"])
dem = np.array(item["dem"])
slope = np.array(item["slope"])

VV = np.stack([VV0, VV1, dem], axis=2)
VH = np.stack([VH0, VH1, dem], axis=2)
    
a = _rescale(VH1 - VH0, 0, .25)

b = _rescale(VV1 - VV0, 0, .25)

w = _rescale(a - b, 0, 1)

r = w*VH0 + (1 - w)*VV0

g = w*VH1 + (1 - w)*VV1

combVVVH = np.stack([r, g, dem], axis=2)

standComb = np.stack([r, g, r], axis=2)

difference = np.stack([VV1 - VV0, VH1 - VH0, dem], axis=2)

# Set a global title size for all subplots
plt.rcParams['axes.titlesize'] = 25
# Plot each channel of the single image
fig, axs = plt.subplots(2, 3, figsize=(24, 16))


axs[0, 0].imshow(standComb)
axs[0, 0].set_title('Standard for Manual')

axs[0, 1].imshow(combVVVH)
axs[0, 1].set_title('Vertical/Horizontal Combination + DEM')

axs[0, 2].imshow(difference)
axs[0, 2].set_title('Difference of Polarizations + DEM')

axs[1, 0].imshow(VV)
axs[1, 0].set_title('Vertical Polarization + DEM')

axs[1, 1].imshow(VH)
axs[1, 1].set_title('Horizontal Polarization + DEM')

axs[1, 2].imshow(ground_truth_mask, cmap='gray')
axs[1, 2].set_title('Ground Truth Mask')

plt.tight_layout()
plt.show()

In [ ]:
item = train_dataset[22]
ground_truth_mask = np.array(item["label"])
VH0 = np.array(item["VH0"])
VH1 = np.array(item["VH1"])
VV0 = np.array(item["VV0"])
VV1 = np.array(item["VV1"])
dem = np.array(item["dem"])
slope = np.array(item["slope"])

VV = np.stack([VV0, VV1, dem], axis=2)
VH = np.stack([VH0, VH1, dem], axis=2)
    
a = _rescale(VH1 - VH0, 0, .25)

b = _rescale(VV1 - VV0, 0, .25)

w = _rescale(a - b, 0, 1)

r = w*VH0 + (1 - w)*VV0

g = w*VH1 + (1 - w)*VV1

combVVVH = np.stack([r, g, dem], axis=2)

standComb = np.stack([r, g, r], axis=2)

difference = np.stack([VV1 - VV0, VH1 - VH0, dem], axis=2)

# Set a global title size for all subplots
plt.rcParams['axes.titlesize'] = 25
# Plot each channel of the single image
fig, axs = plt.subplots(1, 4, figsize=(24, 16))

axs[0].imshow(combVVVH)
axs[0].set_title('V/H Combination + DEM')
axs[0].axis('off')

axs[1].imshow(difference)
axs[1].set_title('Difference of Polarizations + DEM')
axs[1].axis('off')

axs[2].imshow(VV)
axs[2].set_title('Vertical Polarization + DEM')
axs[2].axis('off')

axs[3].imshow(VH)
axs[3].set_title('Horizontal Polarization + DEM')
axs[3].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
plt.imshow(difference)